In [1]:
import litellm
from devtools import debug
from langchain_community.llms.edenai import EdenAI
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

In [2]:
assert litellm.supports_function_calling(model="groq/llama3-70b-8192") is True
assert litellm.supports_function_calling(model="groq/mixtral-8x7b-32768") is True

In [3]:
llm_oai = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm_groq = ChatGroq(model="lLama3-70b-8192")
llm_eden = EdenAI(
    feature="text",
    provider="openai",
    model="gpt-3.5-turbo-instruct",
)

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b



In [1]:


llm_with_tools = llm_oai.bind_tools([add])
llm_with_tools.invoke("What is 45 +  99 ?").tool_calls  # type: ignore # works

# llm_with_tools = llm_eden.bind_tools([add]).tool_calls   # AttributeError: 'EdenAI' object has no attribute 'bind_tools'
# llm_with_tools = llm_groq.bind_tools([add]).tool_calls   AttributeError: 'ChatGroq' object has no attribute 'tool_calls'

AttributeError: 'EdenAI' object has no attribute 'bind_tools'

AttributeError: 'ChatGroq' object has no attribute 'tool_calls'

In [5]:
from langchain_core.pydantic_v1 import BaseModel, Field


class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")


structured_llm = llm_oai.with_structured_output(Joke)
debug(structured_llm.invoke("Tell me a joke about cats"))

/tmp/ipykernel_158360/1147396076.py:10 <module>
    structured_llm.invoke("Tell me a joke about cats"): Joke(
        setup='Why was the cat sitting on the computer?',
        punchline='To keep an eye on the mouse!',
    ) (Joke)


Joke(setup='Why was the cat sitting on the computer?', punchline='To keep an eye on the mouse!')

In [6]:
structured_llm = llm_eden.with_structured_output(Joke)

NotImplementedError: 